In [47]:
import os

os.chdir('/home/yry08/yigedabuliu/lkq/LiteMedSAM')

In [62]:
brats_data_root = '/mnt/sda3/yigedabuliu/lkq/data/Task01_BrainTumour/h5_datasets/'

### 需要将数据集转换为形状为 img:[3,H,W]   gt:[1,H,W]  box:[1,1,4]

In [83]:
# 导入相关包
import torch
from tqdm import tqdm
import numpy as np
import h5py
import matplotlib.pyplot as plt

In [87]:
# 单张截面转换测试

def single_convert(single_img, single_label):
    """
        将单张[H,W]的图像和标签改为合适的尺寸
        single_img: tensor [240,240]
        single_label: tensor [240,240]
    """

    # 将通道数堆叠为3
    single_img = np.stack((single_img,) * 3, axis=-1)
    single_img = np.clip(single_img, 0, 1)

    # 只保留原始标签为1的，即NT区域
    mask = single_label != 1
    single_label[mask] = 0

    # rows, cols = np.where(single_label == 1)
    #
    # # 找到边界
    # top = np.min(rows)
    # bottom = np.max(rows)
    # left = np.min(cols)
    # right = np.max(cols)

    # coords = np.array([[[left, top, right, bottom]]])
    # print(f'single img shape:{single_img.shape}')

    # single_img = np.transpose(single_img, (2, 0, 1))
    single_label = np.expand_dims(single_label, axis=0)

    return single_img, single_label

In [92]:
# 读取并转换数据

save_root = '/mnt/sda3/yigedabuliu/lkq/data/Task01_BrainTumour/npy/'

all_sequence = sorted(os.listdir(brats_data_root))

modal_size = 4

for sequence in tqdm(all_sequence):
    # 读取每个h5文件
    h5_path = brats_data_root + sequence

    h5f = h5py.File(h5_path, 'r')
    image = h5f['image'][:]
    label = h5f['label'][:]

    depth = image.shape[-1]

    # 只保留第1个模态
    cur_modal = 'modal_0'

    for j in range(depth):
        img_f = image[0, :, :, j].copy()
        label_f = label[:, :, j].copy()

        img_f, label_f = single_convert(img_f, label_f)

        # 仅保存非空
        if not np.all(label_f == 0):
            frame_name = sequence.split('.')[0] + '_modal0' + '_f_' + str(j) + '.npy'
            # 保存文件
            img_save_path = save_root + 'imgs/' + frame_name
            label_save_path = save_root + 'gts/' + frame_name

            np.save(img_save_path, img_f)
            np.save(label_save_path, label_f)



100%|███████████████████████████████████████████████████████████████████████████████| 1251/1251 [17:17<00:00,  1.21it/s]
